# Evaluation <a name = "Top"></a>

# Quick Links

<ol>
    <li><a href = #setup>Setup</a></li>
    <li><a href = #plots>Plots</a></li>
</ol>

## Imports

In [ ]:
import os
import shutil

In [ ]:
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import shutil

from enum import Enum

from numpy import array

import ntpath

import copy
import re

## Hilfsfunktionen

### Enum für Trainingsset

In [ ]:
class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Enum für Label-Typ

In [ ]:
class LabelType(Enum):
    ANGULAR = 1
    STEREOGRAPHIC = 2

### Trainingsset-Typ nach String Converter

In [ ]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

### LabelType nach String

In [ ]:
def labeltype_to_string(lt):
    if lt == LabelType.ANGULAR:
        return 'Angular'
    if lt == LabelType.STEREOGRAPHIC:
        return 'Stereographic'
    else:
        print('Unknown LabelType')
        return None

#### Required format of parameters parameter for _model_predict_ (...)

In [ ]:
p = {
    'dataset_to_use':[],
    'model_to_load':[],
    'dataset_name':'combined_dataset',
    'activation_function':[],
    'leaky_ReLU_alpha':[],
    'first_neuron':[],
    'dropout_rate':[],
    'hidden_layers':[],
    'optimizer':[],
    'learning_rate':[],
    'loss_function':[],
    'label_type':[]
}

## Hilfsfunktionen

### Konvertierung ($S_x$, $S_y$) $\rightarrow$ ($\phi$, $\theta$)

In [ ]:
def convert_from_stereographic(sx, sy, r = 1):
    
    l = np.sqrt(sx * sx + sy * sy)
    theta = 90 - 2 * np.degrees(np.arctan(l / 2))
    
    if sx < 0:
        phi = 180 - np.degrees(np.arcsin(sy / l))
        
    elif sx >= 0:
        if sy > 0:
            phi = np.degrees(np.arcsin(sy / l))
        elif sy < 0:
            phi = 360 + np.degrees(np.arcsin(sy / l))
        else:
            #phi1 = np.NaN
            phi = 0
    else:
        print('sx and sy undefined. should not have reached here')

    return phi, theta

### Konvertierung ($\phi$, $\theta$) $\rightarrow$ ($S_x$, $S_y$)

In [ ]:
def convert_from_spheric(phi, theta, r = 1):
    m = 2 * r * np.tan(np.radians((90 - theta) / 2))
    sy = m * np.sin(np.radians(phi))
    sx = m * np.cos(np.radians(phi))
    return sx, sy

### Radians $\rightarrow$ Degree

In [ ]:
def to_degree(angle_in_rad):
    return angle_in_rad * 180 / np.pi

### Degree $\rightarrow$ Radians

In [ ]:
def to_radians(angle_in_deg):
    return angle_in_deg * np.pi / 180

### Sphärische $\rightarrow$ Karthesische Koordinaten

In [ ]:
def spheric_cartesian_polar(phi_d, theta_d):
    x = np.sin(np.radians(90.0 - theta_d)) * np.cos(np.radians(phi_d))
    y = np.sin(np.radians(90.0 - theta_d)) * np.sin(np.radians(phi_d))
    z = np.cos(np.radians(90.0 - theta_d))
    return array([x, y, z])

In [ ]:
def spheric_cartesian_elevation(phi_d, theta_d):
    x = np.cos(np.radians(theta_d)) * np.cos(np.radians(phi_d))
    y = np.cos(np.radians(theta_d)) * np.sin(np.radians(phi_d))
    z = np.sin(np.radians(theta_d))
    return array([x, y, z])

### Length of Vector

In [ ]:
def vectorlength(vector):
    return np.linalg.norm(vector)

### Calculated Angular Error

In [ ]:
def calculate_angular_error(deg_e_phi, deg_e_theta):
    return np.degrees(np.arccos(np.cos(np.radians(deg_e_phi)) * np.cos(np.radians(deg_e_theta))))

### Skalarprodukt

In [ ]:
def myDot(a, b):
    dot = 0;
    it = np.nditer(a, flags=['f_index'])
    for x in it:
        dot = dot + (x * b[it.index])
        
    return dot

### Dot Angular Error

In [ ]:
def dot_angular_error_elevation(predicted_deg_vector, true_deg_vector):    
    c_predicted = spheric_cartesian_elevation(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_elevation(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

def dot_angular_error_polar(predicted_deg_vector, true_deg_vector):
    c_predicted = spheric_cartesian_polar(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_polar(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

## Normierte sphärische Koordinaten

### Normierte $\rightarrow$ Sphärische

In [ ]:
def normalized_to_spheric(n_phi, n_theta):
    phi = n_phi * 180 + 180
    theta = n_theta * 45 + 45
    
    return phi, theta

# Evaluation <a name = "setup"></a>
<p><a href = #Top>Up</a>
<p><a href = #plots>Plots</a>

In [ ]:
run = 'SYNTH'
loss = 'MSE'
dataset_name = '2020-05-28'
net_index = [99, 204, 195]
APPENDIX = ['Angular', 'Normalized', 'Stereographic']
_note = ['', '', '_Custom-MAE']

eval_dir = '..\\output\\{}_Regression_{}\\Graphical_Evaluation\\'.format(run, loss)

if(not os.path.exists(eval_dir)):
    os.makedirs(eval_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(eval_dir))

In [ ]:
import pickle

net_results = [None, None, None]

for i in range(3):
    with open(eval_dir + '{}_Net{}_{}{}_Results.pickle'.format(dataset_name, net_index[i], APPENDIX[i], _note[i]), "rb") as fp:   # Unpickling
        net_results[i] = pickle.load(fp)

In [ ]:
for i in range(3):
    print(net_results[i])
    print()
    print()
    print('---------------------------------------------------------------------')
    print()
    print()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.text as text
from matplotlib.pyplot import figure

from matplotlib import cm

import tikzplotlib
%matplotlib inline

Enable_Plotting = True

# Plots <a name = "plots">
<p></a><a href = #Top>Up</a>
<p><a href = #setup>Setup</a>

In [ ]:
if Enable_Plotting:
    interesting_cases = [0, 3, 4, 5]
    interesting_cases = [0, 3, 4, 5]
    
    net_len = len(net_results)
    net_len = 3
    
    net_df = [[], [], []]
    
    for net_idx in range(net_len):
        net = net_results[net_idx]
        for run_idx in interesting_cases:
            df = net[run_idx][1]
            net_df[net_idx].append(df['dot_angular_err_elevation'])
            #print(df['dot_angular_err_elevation'])

##  Plot Box-Whisker-Diagram

In [ ]:
def do_Plot(title, eval_file, save = True):
    fig, ax = plt.subplots(figsize = (15,15))
    
    s_s_df = (net_df[0][0], net_df[1][0], net_df[2][0])
    s_r_df = (net_df[0][1], net_df[1][1], net_df[2][1])
    r_r_df = (net_df[0][2], net_df[1][2], net_df[2][2])
    m_r_df = (net_df[0][3], net_df[1][3], net_df[2][3])
    
    s_s_pos = [0.5, 3.5, 6.5]
    s_r_pos = [1, 4, 7]
    r_r_pos = [1.5, 4.5, 7.5]
    m_r_pos = [2, 5, 8]
    
    w = [0.3, 0.3, 0.3]
    
    cmap = cm.get_cmap('tab20c')
    #colours = [cmap(0.), cmap(0.05), cmap(0.1), cmap(0.15), cmap(0.2), cmap(0.25), cmap(0.3), cmap(0.35)]
    colours = [cmap(0.1), cmap(0.3), cmap(0.5), cmap(0.7)]
    
    p1 = plt.boxplot(s_s_df, positions = s_s_pos, widths = w, showmeans = True, meanline = True, patch_artist = True, 
                         boxprops = dict(facecolor = colours[0]))
    p2 = plt.boxplot(s_r_df, positions = s_r_pos, widths = w, showmeans = True, meanline = True, patch_artist = True, 
                     boxprops = dict(facecolor = colours[1]))
    p3 = plt.boxplot(r_r_df, positions = r_r_pos, widths = w, showmeans = True, meanline = True, patch_artist = True, 
                     boxprops = dict(facecolor = colours[2]))
    p4 = plt.boxplot(m_r_df, positions = m_r_pos, widths = w, showmeans = True, meanline = True, patch_artist = True, 
                     boxprops = dict(facecolor = colours[3]))
    
    ticks = [1.25, 4.25, 7.25]
    lbls = [
        'Net_p,t', 
        'Net_|p|,|t|', 
        'Net_sx,sy'
    ]
    plt.xticks(ticks, lbls)

    #plt.setp(ax.get_xticklabels(), rotation = 'vertical')
    plt.ylabel('Distribution of\nthe angular error')
    plt.title(title)
    ax.grid(True, ls = '--', axis = 'y')
    
    ax.legend([p1["boxes"][0], p2["boxes"][0], p3["boxes"][0], p4["boxes"][0]], ['Train: synth\nTest: synth', 'Train: synth\nTest: real', 'Train: real\nTest: real', 'Train: mixed\nTest: real'], loc='upper right')
    
    plots = [p1, p2, p3, p4]
    for p in plots:
        caps = p['caps']
        for cap in caps:
            cap.set(xdata = cap.get_xdata() + (-.075, +.075))
    
    #start, end = ax.get_ylim()
    ax.yaxis.set_ticks(np.arange(-10, 170, 10))
    
    for o in fig.findobj(text.Text):
        o.set_fontstyle('italic')
    
    if(save):
        tikzplotlib.save('{}.tex'.format(eval_file))
        plt.savefig('{}.png'.format(eval_file), format = 'png', bbox_inches = "tight", dpi = 300)

    plt.show()

### Show Plot

In [ ]:
if Enable_Plotting:
    tsts_plot = do_Plot('Angular Prediction Error Distribution', eval_dir + 'Box-Whisker-Plot')